In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import tempfile, os, zipfile
from PIL import Image
import requests
from matplotlib import pyplot as plt
#mport squarify
import random
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import classification_report

from os import listdir
from os.path import isfile, join

In [2]:
import requests
from urllib.parse import urlencode

base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?'
public_key = 'https://disk.yandex.com.am/d/1mFfgWQxggpeUQ'
final_url = base_url + urlencode(dict(public_key=public_key))
response = requests.get(final_url)

download_url = response.json()['href']

download_response = requests.get(download_url)


file = tempfile.TemporaryFile()
file.write(download_response.content)
fzip = zipfile.ZipFile(file)
fzip.extractall('./TJ_data')

file.close()
fzip.close()

# https://www.kaggle.com/datasets/balabaskar/tom-and-jerry-image-classification/download?datasetVersionNumber=3

In [3]:
listdir("./TJ_data") # Tom&Jerry_dataset
listdir("./TJ_data/Tom&Jerry_dataset")

['archive.zip']

In [4]:
fzip = zipfile.ZipFile("./TJ_data/Tom&Jerry_dataset/archive.zip")
fzip.extractall('./TJ_data/dataset')

fzip.close()

In [5]:
dataset_dir = "./TJ_data/dataset"
tom_dir = "./TJ_data/dataset/tom_and_jerry/tom_and_jerry/tom"
jerry_dir = "./TJ_data/dataset/tom_and_jerry/tom_and_jerry/jerry"
tj_0_dir = "./TJ_data/dataset/tom_and_jerry/tom_and_jerry/tom_jerry_0"
tj_1_dir = "./TJ_data/dataset/tom_and_jerry/tom_and_jerry/tom_jerry_1"

print(listdir(dataset_dir))


['challenges.csv', 'ground_truth.csv', 'tom_and_jerry']


In [6]:
RANDOM_STATE = 42

In [7]:
import os
import numpy as np
from PIL import Image

folder_path = './TJ_data/dataset/tom_and_jerry/tom_and_jerry/'


def images_to_array(folder_path, class_name):

    image_files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

    data = []
    target= []
    for image_file in image_files:
        image_path = os.path.join(folder_path, image_file)

        img = Image.open(image_path)

        img = img.resize((224, 224))

        img_array = np.array(img, dtype=float) / 255.0  # Нормализация значений пикселей к диапазону [0, 1]
        img_vector = img_array.flatten()

        data.append(img_vector)
        target.append(class_names.index(class_name))


    result_array = np.array(data)
    target_array = np.array(target)
    return result_array, target_array

folder_path = './TJ_data/dataset/tom_and_jerry/tom_and_jerry/'
class_names =  ['jerry', 'tom', 'tom_jerry_0', 'tom_jerry_1']
result_array = None
target_result_array = None


for class_name in class_names:
    class_directory = os.path.join(folder_path, class_name)
    class_array, target_array = images_to_array(class_directory, class_name)

    if result_array is None:
        result_array = class_array
    else:
        result_array = np.concatenate((result_array, class_array))
    if target_result_array is None:
        target_result_array=target_array
    else:
        target_result_array=np.concatenate((target_result_array, target_array))

print(result_array.shape)
print(target_result_array.shape)

(5478, 150528)
(5478,)


In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(result_array,target_result_array,
                                                    test_size=0.25,
                                                    random_state=1984)

Код ниже должен был быть итеративным от 1 до размерности массива, но к сожалению это безумно долго. Поэтому ручками

In [12]:
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

accuracy_scores = []
precision_scores=[]
recall_scores=[]
f1_scores=[]
component_range = range(1, 1000, 100)
for n_components in component_range:
    pca = PCA(n_components=n_components)
    X_train_pca = pca.fit_transform(X_train)
    model = SVC(kernel='rbf', class_weight ='balanced')
    model.fit(X_train_pca, y_train)
    X_test_pca = pca.transform(X_test)
    y_pred = model.predict(X_test_pca)
    accuracy = accuracy_score(y_test, y_pred)
    print("accuracy:",n_components, accuracy)
    accuracy_scores.append(accuracy)
    precision = precision_score(y_test, y_pred, average='weighted')
    print("precision:", n_components, precision)
    precision_scores.append(precision)
    recall = recall_score(y_test, y_pred, average='weighted')
    print("recall:", n_components, recall)
    recall_scores.append(recall)
    f1 = f1_score(y_test, y_pred, average='weighted')
    print("f1:",n_components, f1)
    f1_scores.append(f1)
best_index = np.argmax(accuracy_scores)
best_n_components = component_range[best_index]
print("Наилучшее количество компонент для accuracy:", best_n_components)
best_index = np.argmax(accuracy_scores)
best_n_components = component_range[best_index]
print("Наилучшее количество компонент для precision:", best_n_components)
best_index = np.argmax(accuracy_scores)
best_n_components = component_range[best_index]
print("Наилучшее количество компонент для recall:", best_n_components)
best_index = np.argmax(accuracy_scores)
best_n_components = component_range[best_index]
print("Наилучшее количество компонент для f1:", best_n_components)


accuracy: 1 0.32408759124087594
precision: 1 0.34209740333717314
recall: 1 0.32408759124087594
f1: 1 0.32879609216717554
accuracy: 101 0.7124087591240876
precision: 101 0.7128149626833584
recall: 101 0.7124087591240876
f1: 101 0.712473584202829
accuracy: 201 0.7299270072992701
precision: 201 0.7321979585974646
recall: 201 0.7299270072992701
f1: 201 0.7307006184313248
accuracy: 301 0.7350364963503649
precision: 301 0.7379214081149076
recall: 301 0.7350364963503649
f1: 301 0.73607047967608
accuracy: 401 0.7416058394160584
precision: 401 0.74541240293817
recall: 401 0.7416058394160584
f1: 401 0.7429171577435059
accuracy: 501 0.743065693430657
precision: 501 0.7472094116993546
recall: 501 0.743065693430657
f1: 501 0.744450985334619
accuracy: 601 0.7467153284671533
precision: 601 0.7507761663367545
recall: 601 0.7467153284671533
f1: 601 0.7480997737300276
accuracy: 701 0.7474452554744525
precision: 701 0.7515848179929534
recall: 701 0.7474452554744525
f1: 701 0.7488555815096005
accuracy: 80

In [13]:
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

accuracy_scores = []
precision_scores=[]
recall_scores=[]
f1_scores=[]
component_range = range(1000, 2000, 100)
for n_components in component_range:
    pca = PCA(n_components=n_components)
    X_train_pca = pca.fit_transform(X_train)
    model = SVC(kernel='rbf', class_weight ='balanced')
    model.fit(X_train_pca, y_train)
    X_test_pca = pca.transform(X_test)
    y_pred = model.predict(X_test_pca)
    accuracy = accuracy_score(y_test, y_pred)
    print("accuracy:",n_components, accuracy)
    accuracy_scores.append(accuracy)
    precision = precision_score(y_test, y_pred, average='weighted')
    print("precision:", n_components, precision)
    precision_scores.append(precision)
    recall = recall_score(y_test, y_pred, average='weighted')
    print("recall:", n_components, recall)
    recall_scores.append(recall)
    f1 = f1_score(y_test, y_pred, average='weighted')
    print("f1:",n_components, f1)
    f1_scores.append(f1)
best_index = np.argmax(accuracy_scores)
best_n_components = component_range[best_index]
print("Наилучшее количество компонент для accuracy:", best_n_components)
best_index = np.argmax(accuracy_scores)
best_n_components = component_range[best_index]
print("Наилучшее количество компонент для precision:", best_n_components)
best_index = np.argmax(accuracy_scores)
best_n_components = component_range[best_index]
print("Наилучшее количество компонент для recall:", best_n_components)
best_index = np.argmax(accuracy_scores)
best_n_components = component_range[best_index]
print("Наилучшее количество компонент для f1:", best_n_components)


accuracy: 1000 0.7467153284671533
precision: 1000 0.7515374687300141
recall: 1000 0.7467153284671533
f1: 1000 0.748227297916372
accuracy: 1100 0.7481751824817519
precision: 1100 0.7533219080968533
recall: 1100 0.7481751824817519
f1: 1100 0.7497583902230195
accuracy: 1200 0.7503649635036497
precision: 1200 0.755198569307096
recall: 1200 0.7503649635036497
f1: 1200 0.7518776476422508
accuracy: 1300 0.7489051094890511
precision: 1300 0.7542182534209849
recall: 1300 0.7489051094890511
f1: 1300 0.7505246051927658
accuracy: 1400 0.7496350364963503
precision: 1400 0.7552108171865096
recall: 1400 0.7496350364963503
f1: 1400 0.7512895479213212
accuracy: 1500 0.7496350364963503
precision: 1500 0.754972655115491
recall: 1500 0.7496350364963503
f1: 1500 0.7512520709432694
accuracy: 1600 0.7496350364963503
precision: 1600 0.7552108171865096
recall: 1600 0.7496350364963503
f1: 1600 0.7512895479213212
accuracy: 1700 0.7503649635036497
precision: 1700 0.755800190534523
recall: 1700 0.7503649635036497
